<a href="https://colab.research.google.com/github/pruthvirajv/Transformer-Neural-Network/blob/main/MachineTranslation/torchtext_translation_tutorial_with_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline

In [ ]:
!python -m pip install --upgrade pip
!python -m pip install torchtext==0.6.0
!python -m pip install einops

In [1]:
from einops import rearrange

In [4]:
!python -m pip install spacy

In [5]:
!python -m spacy download en
!python -m spacy download fr
!python -m spacy download de

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 65.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
⚠ As of spaCy v3.0, shortcuts like 'fr' are deprecated. Please use the
full pipeline package name 'fr_core_news_sm' instead.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 43.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependenc


Language Translation with TorchText
===================================

This tutorial shows how to use several convenience classes of ``torchtext`` to preprocess
data from a well-known dataset containing sentences in both English and German and use it to
train a sequence-to-sequence model with attention that can translate German sentences
into English.

- [Link to Original PyTorch Tutorial](https://pytorch.org/tutorials/beginner/torchtext_translation_tutorial.html?highlight=transformer)
- [Link to Andrew Peng's language translation implementation](https://github.com/andrewpeng02/transformer-translation)
- [Interesting info about sorting sentences](https://towardsdatascience.com/understanding-transformers-the-programming-way-f8ed22d112b2)

It is based off of
`this tutorial <https://github.com/bentrevett/pytorch-seq2seq/blob/master/3%20-%20Neural%20Machine%20Translation%20by%20Jointly%20Learning%20to%20Align%20and%20Translate.ipynb>`__
from PyTorch community member `Ben Trevett <https://github.com/bentrevett>`__
and was created by `Seth Weidman <https://github.com/SethHWeidman/>`__ with Ben's permission.

## Dataset
----------------
In this tutorial we'll use the `Multi30k dataset <https://github.com/multi30k/dataset>`__, which contains about 30,000 sentences (averaging about 13 words in length) in both English and German.

In [2]:
import spacy
import torchtext
from torchtext.data import Field, BucketIterator

In [ ]:
!python -m spacy download de
!python -m spacy download en

In [3]:
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator



In [ ]:
!python -m spacy download de_core_news_sm  # Download German model
!python -m spacy download en_core_web_sm  # Download German model

In [5]:
import spacy
import torchtext
from torchtext.data import Field, BucketIterator



# SRC = Field(tokenize = "spacy",
#             tokenizer_language="de",
#             init_token = '<sos>',
#             eos_token = '<eos>',
#             lower = True)

# Load the German spaCy model explicitly
spacy_de = spacy.load("de_core_news_sm")

def tokenize_de(text):
    """
    Tokenizes German text using the loaded spaCy model.
    """
    return [tok.text for tok in spacy_de.tokenizer(text)]

SRC = Field(tokenize = tokenize_de,  # Use custom tokenizer function
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = True)



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 137.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [8]:
# TRG = Field(tokenize = "spacy",
#             tokenizer_language="en",
#             init_token = '<sos>',
#             eos_token = '<eos>',
#             lower = True)
TRG = Field(tokenize = "spacy",
            tokenizer_language="de_core_news_sm",  # Use full model name
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = True)

TRG = Field(tokenize = "spacy",
            tokenizer_language="en_core_web_sm",  # Use full model name
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = True)



  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.1/en_core_web_sm-3.7.1-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install torchdata

In [13]:
import ssl
import torch
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator
import os

# Define a custom SSL context with relaxed verification
# (Use with caution! Not recommended for production environments)
ssl_context = ssl.create_default_context()
ssl_context.check_hostname = False  # Disable hostname verification
ssl_context.verify_mode = ssl.CERT_NONE  # Disable certificate verification


# Specify the root directory where the dataset should be downloaded
root = './my_data'

# Create the root directory if it doesn't exist
os.makedirs(root, exist_ok=True)

# Download and split the dataset using the custom SSL context
train_data, valid_data, test_data = Multi30k.splits(
    root=root,
    exts=('.de', '.en'),
    fields=(SRC, TRG),
    ssl_context=ssl_context  # Pass the custom SSL context
)


#train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), fields = (SRC, TRG))



FileNotFoundError: [Errno 2] No such file or directory: './my_data/multi30k/train.de'

In [ ]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

## What does our data look like?
Torchtext preprocesses our data to give us a mapping from our source language to the target language and back.

In [ ]:
# Printing a list of tokens mapping integer to strings
print(SRC.vocab.itos)
# Printing a dict mapping tokens to indices
print(SRC.vocab.stoi)
# Printing the index of an actual word
print(SRC.vocab.stoi['ein'])

``BucketIterator``
----------------
The last ``torchtext`` specific feature we'll use is the ``BucketIterator``,
which is easy to use since it takes a ``TranslationDataset`` as its
first argument. Specifically, as the docs say:
Defines an iterator that batches examples of similar lengths together.
Minimizes amount of padding needed while producing freshly shuffled
batches for each new epoch. See pool for the bucketing procedure used.



In [ ]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    device = device)

Defining our ``nn.Module`` and ``Optimizer``
----------------
We define our ``nn.Module`` using the nn.Transformer module implemented in PyTorch.
In order to use the Transformer


In [ ]:
import random
from typing import Tuple

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import Tensor

import math

INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)

ENC_EMB_DIM = 32
DEC_EMB_DIM = 32
ENC_HID_DIM = 64
DEC_HID_DIM = 64
ATTN_DIM = 8
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

# Source: https://pytorch.org/tutorials/beginner/transformer_tutorial.html
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=100):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

class TransformerModel(nn.Module):
    def __init__(self):
        super(TransformerModel, self).__init__()
        self.src_embedding = nn.Embedding(INPUT_DIM, ENC_EMB_DIM)
        self.tgt_embedding = nn.Embedding(INPUT_DIM, ENC_EMB_DIM)
        self.transformer = nn.Transformer(nhead=8, num_encoder_layers=2, d_model=ENC_EMB_DIM)
        self.linear = nn.Linear(ENC_EMB_DIM, OUTPUT_DIM)
        pos_dropout = 0.1
        max_seq_length = 128
        self.pos_enc = PositionalEncoding(ENC_EMB_DIM, pos_dropout, max_seq_length)

    def forward(self, src, tgt, teacher_forcing_ratio=0.5, src_key_padding_mask=None, tgt_key_padding_mask=None, memory_key_padding_mask=None, tgt_mask=None):
        # TODO: Investigate masks, positional encoding, understand Rearrange(), debug model output (output has negative numbers for some reason)
        # Original src shape: (sentence length=24?, batch_size=128)
        # Original tgt shape: (sentence length=24?, batch_size=128)
        # Transformer expects: (sentence length=24, batch_size=128, embedding_size=128)
        src_emb = self.pos_enc(self.src_embedding(src) * math.sqrt(ENC_EMB_DIM))
        tgt_emb = self.pos_enc(self.tgt_embedding(tgt) * math.sqrt(ENC_EMB_DIM))
        out = self.transformer(src_emb, tgt_emb, tgt_mask=tgt_mask, src_key_padding_mask=src_key_padding_mask, tgt_key_padding_mask=tgt_key_padding_mask, memory_key_padding_mask=memory_key_padding_mask)
        out = self.linear(out)
        return out

model = TransformerModel().to(device)
for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_normal_(p)


optimizer = optim.Adam(model.parameters(), betas=(0.9, 0.98))


def count_parameters(model: nn.Module):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f'The model has {count_parameters(model):,} trainable parameters')

Note: when scoring the performance of a language translation model in
particular, we have to tell the ``nn.CrossEntropyLoss`` function to
ignore the indices where the target is simply padding.



In [ ]:
# Checking index of special tokens
PAD_IDX = TRG.vocab.stoi['<pad>']
SOS_IDX = TRG.vocab.stoi['<sos>']
EOS_IDX = TRG.vocab.stoi['<eos>']
UNK_IDX = TRG.vocab.stoi['<unk>']
print('pad index:', PAD_IDX)
print('sos index:', SOS_IDX)
print('eos index:', EOS_IDX)
print('unk index:', UNK_IDX)

criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

## Training and Evaluation


In [ ]:
def gen_nopeek_mask(length):
    mask = rearrange(torch.triu(torch.ones(length, length)) == 1, 'h w -> w h')
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))

    return mask

In [ ]:
def example_translate(model, example_sentence_src):
    # Translate example sentence
    example_tensor_src = string_to_indices(SRC, example_sentence_src).view(-1, 1)
    example_sentence_tgt = '<sos>'
    example_tensor_tgt = string_to_indices(TRG, example_sentence_tgt).view(-1, 1)
    src = example_tensor_src.to(device)
    tgt = example_tensor_tgt.to(device)

    print('----------Translating--------------')
    for i in range(128):
        print('Src:', src)
        print('Tgt:', tgt)
        src_key_padding_mask = src == PAD_IDX
        tgt_key_padding_mask = tgt == PAD_IDX
        memory_key_padding_mask = src_key_padding_mask.clone()
        src_key_padding_mask = rearrange(src_key_padding_mask, 'n s -> s n')
        tgt_key_padding_mask = rearrange(tgt_key_padding_mask, 'n s -> s n')
        memory_key_padding_mask = rearrange(memory_key_padding_mask, 'n s -> s n')
        tgt_mask = gen_nopeek_mask(tgt.shape[0]).to('cuda')
        print('Tgt mask:', tgt_mask)

        output = model(src, tgt, 0, src_key_padding_mask=src_key_padding_mask, tgt_key_padding_mask=tgt_key_padding_mask, memory_key_padding_mask=memory_key_padding_mask, tgt_mask=tgt_mask) #turn off teacher forcing

        print('Output:', output)
        # TODO: Check that the argmax line is correct
        output_index = torch.argmax(output, dim=2)[-1].item()
        output_word = TRG.vocab.itos[output_index]
        example_sentence_tgt = example_sentence_tgt + ' ' + output_word
        print('Translated sentence so far:', example_sentence_tgt)
        example_tensor_tgt = string_to_indices(TRG, example_sentence_tgt).view(-1, 1)
        tgt = example_tensor_tgt.to(device)
        if output_word == '<eos>':
            break
    print('-----------Finished Translating--------------')


In [ ]:
import math
import time

def indices_to_string(LANGUAGE, batch):
    words_list = []
    for sentence in batch.transpose(1, 0):
        sentence_list = sentence.tolist()
        words = []
        for index in sentence_list:
            word = LANGUAGE.vocab.itos[index]
            words.append(word)
        words_list.append(words)
    return words_list

def string_to_indices(LANGUAGE, sentence):
    words = sentence.split()
    indices = []
    for word in words:
        if word in LANGUAGE.vocab.stoi:
            index = LANGUAGE.vocab.stoi[word]
            indices.append(index)
        else:
            index = LANGUAGE.vocab.stoi['<unk>']
            indices.append(index)
    result = torch.tensor(indices)
    return result

def train(model: nn.Module,
          iterator: BucketIterator,
          optimizer: optim.Optimizer,
          criterion: nn.Module,
          clip: float):

    model.train()

    epoch_loss = 0

    for _, batch in enumerate(iterator):

        src = batch.src
        tgt = batch.trg

        optimizer.zero_grad()

        # Original src shape: (sentence length=24, batch_size=128)
        # Transformer expects: (sentence length=24, batch_size=128, embedding_size=128)
        src_key_padding_mask = src == PAD_IDX
        tgt_key_padding_mask = tgt == PAD_IDX
        memory_key_padding_mask = src_key_padding_mask.clone()
        src_key_padding_mask = rearrange(src_key_padding_mask, 'n s -> s n')
        tgt_key_padding_mask = rearrange(tgt_key_padding_mask, 'n s -> s n')
        memory_key_padding_mask = rearrange(memory_key_padding_mask, 'n s -> s n')
        tgt_sentence_len = tgt.shape[0] - torch.sum(tgt_key_padding_mask, axis=1)
        tgt_inp, tgt_out = tgt[:-1, :], tgt[1:, :]
        tgt_key_padding_mask = tgt_key_padding_mask[:, :-1]
        tgt_mask = gen_nopeek_mask(tgt_inp.shape[0]).to('cuda')
        output = model(src, tgt_inp, src_key_padding_mask=src_key_padding_mask, tgt_key_padding_mask=tgt_key_padding_mask, memory_key_padding_mask=memory_key_padding_mask, tgt_mask=tgt_mask)
        from_one_hot = torch.argmax(output, dim=2)
        # output shape: (sentence length=24, batch_size=128, vocab=5893)
        # Original tgt shape: (sentence length=24, batch_size=128)

        output = output.view(-1, output.shape[-1])
        tgt_out = tgt_out.view(-1)

        loss = criterion(output, tgt_out)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)



def evaluate(model: nn.Module,
             iterator: BucketIterator,
             criterion: nn.Module):

    print('Evaluating')
    model.eval()

    epoch_loss = 0

    with torch.no_grad():

        for _, batch in enumerate(iterator):

            src = batch.src
            tgt = batch.trg

            src_key_padding_mask = src == PAD_IDX
            tgt_key_padding_mask = tgt == PAD_IDX
            memory_key_padding_mask = src_key_padding_mask.clone()
            src_key_padding_mask = rearrange(src_key_padding_mask, 'n s -> s n')
            tgt_key_padding_mask = rearrange(tgt_key_padding_mask, 'n s -> s n')
            memory_key_padding_mask = rearrange(memory_key_padding_mask, 'n s -> s n')
            tgt_mask = gen_nopeek_mask(tgt.shape[0]).to('cuda')
            output = model(src, tgt, 0, src_key_padding_mask=src_key_padding_mask, tgt_key_padding_mask=tgt_key_padding_mask, memory_key_padding_mask=memory_key_padding_mask, tgt_mask=tgt_mask) #turn off teacher forcing
            from_one_hot = torch.argmax(output, dim=2)
            #print(src.shape, output.shape, from_one_hot.shape, tgt.shape)

            output = output[1:].view(-1, output.shape[-1])
            #print('Src:', src.transpose(1, 0))
            #print('Predicted:', from_one_hot.transpose(1, 0))
            #print('Target:', tgt.transpose(1, 0))
            src_words = indices_to_string(SRC, src)
            predicted_words = indices_to_string(TRG, from_one_hot)
            tgt_words = indices_to_string(TRG, tgt)
            #for i, src in enumerate(src_words):
            #    print('----------------------------------')
            #    print(' '.join(src))
            #    print(' '.join(predicted_words[i]))
            #    print(' '.join(tgt_words[i]))

            tgt = tgt[1:].view(-1)
            loss = criterion(output, tgt)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def epoch_time(start_time: int,
               end_time: int):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


In [ ]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

In [ ]:
test_loss = evaluate(model, test_iterator, criterion)

In [ ]:
example_sentence_src = '<sos> der himmel ist blau <eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>'
#example_sentence_src = '<sos> viele menschen haben sich versammelt , um etwas zu sehen , das nicht auf dem foto ist . <eos> <pad> <pad> <pad> <pad>'
example_translate(model, example_sentence_src)